In [148]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import geopandas
from sklearn import linear_model
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
import matplotlib.pyplot as plt
import warnings
from pysal.lib import cg as geometry
from pysal.lib import weights
warnings.filterwarnings("ignore")
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points

In [149]:
df = pd.read_csv('co_properties.csv.gz', compression='gzip', header=0,    sep=',', quotechar='"', error_bad_lines=False)

- The most reliable Python library to manage spatial data is geopandas. Unfortunately, the instalation of geopandas is not as trivial as the other Python librarys we have seen. For this reason, I leave a blog with a simple way of installing [geopandas](https://medium.com/analytics-vidhya/fastest-way-to-install-geopandas-in-jupyter-notebook-on-windows-8f734e11fa2b)

- Data preprocesing

In [150]:
df=df[df['operation_type']=='Venta']
df=df[df['property_type']=='Casa']
df=df[df['currency']=='COP']
df=df[df['l3']=='Bogotá D.C']
df=df[[ 'price', 'rooms', 'bathrooms', 'surface_total', 'surface_covered','lat','lon']]
df=df.dropna()

In [151]:
df = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.lon, df.lat))
df=df.set_crs(epsg=4326, inplace=True)

- Let's see if we can use the distance to the center of the city as a feature in our model

In [132]:
lat=[4.61083]
lon=[-74.070278]
colp=pd.DataFrame({'lat':lat,'lon':lon})
colp = geopandas.GeoDataFrame(colp , geometry=geopandas.points_from_xy(colp.lon, colp.lat))
df=df.reset_index(drop=True)
colp=colp.set_crs(epsg=4326, inplace=True)
colp=colp.to_crs(epsg=3310)
df=df.to_crs(epsg=3310)
dis=[]
for i in tqdm(df.index):
    dis.append(colp.distance(df['geometry'][i])[0])

100%|██████████| 1132/1132 [00:00<00:00, 11546.44it/s]


In [133]:
df['dis_centro']=dis

- Import shapefile with geopandas

In [161]:
estaciones=geopandas.read_file('estaciones.zip')
estaciones=estaciones.to_crs(epsg=4326)
df=df.reset_index(drop=True)
puntos = estaciones['geometry'].tolist()
puntos = MultiPoint(puntos)

- Let´s find the distance of each property to the nearest transmilenio station

In [172]:
n=nearest_points(df['geometry'][0],puntos)[1]

In [178]:
estaciones[estaciones['geometry'] == n].index[0]

510

In [191]:
(estaciones.iloc[[510]].distance(df['geometry'][0]))[510]

0.0013144006860216198

In [193]:
distancia_transmi=[]
for i in tqdm(df.index):
    n=nearest_points(df['geometry'][0],puntos)[1]
    y=estaciones[estaciones['geometry'] == n].index[0]
    distancia_transmi.append((estaciones.iloc[[y]].distance(df['geometry'][0]))[y])
    
    

100%|██████████| 1132/1132 [03:18<00:00,  5.70it/s]


In [ ]:
df['distancia_transmi']=distancia_transmi

## **lasso CV**

- common notation l1=Lasso l2=ridge

In [134]:
df=df.drop(columns=['geometry'])

In [135]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['price']),df['price'], test_size=0.30,
                                                    random_state=289988888,
                                                    shuffle=True)

In [136]:
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
lasso_params = {'alpha':np.logspace(-8, 8, 100)}
lasso= linear_model.Lasso()

            
model_cv = GridSearchCV(estimator = lasso, 
                        param_grid = lasso_params, 
                        scoring='neg_mean_squared_error', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True
                       ,n_jobs=10)      
bla=model_cv.fit(x_train, y_train)  

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [137]:
bla.best_estimator_

Lasso(alpha=1.9630406500402725e-07)

In [138]:
mse(y_test, bla.predict(x_test))

1.00956155552173e+18

In [139]:
mse(y_train, bla.predict(x_train))

4.626427938004997e+17

In [140]:
pd.DataFrame(bla.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.004968,0.002594,0.001200,0.000980,0.0,{'alpha': 1e-08},-1.000769e+19,-4.521354e+17,-5.602648e+17,-5.446985e+17,...,-2.389736e+18,3.809516e+18,3,-4.101373e+17,-4.665496e+17,-4.406621e+17,-4.445249e+17,-4.839554e+17,-4.491658e+17,2.501835e+16
1,0.006249,0.007653,0.000000,0.000000,0.0,{'alpha': 1.4508287784959402e-08},-1.000769e+19,-4.521354e+17,-5.602648e+17,-5.446985e+17,...,-2.389736e+18,3.809516e+18,7,-4.101373e+17,-4.665496e+17,-4.406621e+17,-4.445249e+17,-4.839554e+17,-4.491658e+17,2.501835e+16
2,0.011168,0.006149,0.000399,0.000798,0.0,{'alpha': 2.104904144512022e-08},-1.000769e+19,-4.521354e+17,-5.602648e+17,-5.446985e+17,...,-2.389736e+18,3.809516e+18,10,-4.101373e+17,-4.665496e+17,-4.406621e+17,-4.445249e+17,-4.839554e+17,-4.491658e+17,2.501835e+16
3,0.006116,0.005820,0.002234,0.002289,0.0,{'alpha': 3.053855508833412e-08},-1.000769e+19,-4.521354e+17,-5.602648e+17,-5.446985e+17,...,-2.389736e+18,3.809516e+18,9,-4.101373e+17,-4.665496e+17,-4.406621e+17,-4.445249e+17,-4.839554e+17,-4.491658e+17,2.501835e+16
4,0.013899,0.003446,0.000000,0.000000,0.0,{'alpha': 4.4306214575838774e-08},-1.000769e+19,-4.521354e+17,-5.602648e+17,-5.446985e+17,...,-2.389736e+18,3.809516e+18,13,-4.101373e+17,-4.665496e+17,-4.406621e+17,-4.445249e+17,-4.839554e+17,-4.491658e+17,2.501835e+16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.004420,0.004647,0.001604,0.003208,22570197.196339,{'alpha': 22570197.19633926},-1.372246e+19,-5.293196e+17,-7.004913e+17,-6.607538e+17,...,-3.214923e+18,5.254478e+18,96,-4.909635e+17,-5.670539e+17,-5.289538e+17,-5.352102e+17,-5.828938e+17,-5.410150e+17,3.198807e+16
96,0.001610,0.003220,0.003624,0.003688,32745491.628777,{'alpha': 32745491.628777318},-1.403229e+19,-5.290995e+17,-7.054859e+17,-6.600150e+17,...,-3.277268e+18,5.378235e+18,97,-4.918218e+17,-5.677285e+17,-5.296472e+17,-5.359985e+17,-5.836450e+17,-5.417682e+17,3.194510e+16
97,0.002418,0.003906,0.000000,0.000000,47508101.621028,{'alpha': 47508101.621028125},-1.449050e+19,-5.290452e+17,-7.130617e+17,-6.596746e+17,...,-3.369839e+18,5.561075e+18,98,-4.936282e+17,-5.691485e+17,-5.311065e+17,-5.376578e+17,-5.852259e+17,-5.433534e+17,3.185504e+16
98,0.004421,0.003889,0.002008,0.003107,68926121.043497,{'alpha': 68926121.04349709},-1.513057e+19,-5.295245e+17,-7.247494e+17,-6.607206e+17,...,-3.499998e+18,5.816063e+18,99,-4.971051e+17,-5.721370e+17,-5.341787e+17,-5.411506e+17,-5.885541e+17,-5.466251e+17,3.176796e+16


## **Ridge CV**

In [118]:
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
Ridge_params = {'alpha':np.logspace(-8, 8, 100)}
Ridge= linear_model.Ridge()

            
model2_cv = GridSearchCV(estimator = Ridge, 
                        param_grid = Ridge_params, 
                        scoring='neg_mean_squared_error', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True
                       ,n_jobs=10)      
bla2=model2_cv.fit(x_train, y_train)  

Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [120]:
bla2.best_estimator_

Ridge(alpha=9111.627561154906)

In [121]:
mse(y_test, bla2.predict(x_test))

1.0149229389884774e+18

In [122]:
mse(y_train, bla2.predict(x_train))

2.0961153848592276e+18

## **General function**

In [123]:
def entrenar_modelo(modelo,grid,data,nombre):

    x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=['malignant']),data['malignant'], test_size=0.30,
                                                    random_state=28888888,
                                                    shuffle=True)

    gsearch = GridSearchCV(estimator = modelo,
                           param_grid = grid,                         
                           scoring='neg_mean_squared_error', 
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)
    model=gsearch.fit(x_train,y_train)
    mejor_estimador=bla2.best_estimator_
    in_sample=mse(y_test, bla2.predict(x_test))
    out_of_sample=mse(y_train, bla2.predict(x_train))

   

    
    return model,in_sample,mejor_estimador